## Evaluating ConcolicGrammarFuzzer

In this chapter, we will evaluate `ConcolicGrammarFuzzer`, which is our concolic fuzzer that uses grammars.

Our target will be a basic program that mimics an SQL database. We will fuzz this program using the `INVENTORY_GRAMMAR`, both with `ConcolicGrammarFuzzer` and `GrammarCoverageFuzzer`, then we will compare the success of these fuzzers. Namely, we will consider **line coverage**, **branch coverage** and **program depth** reached.

In [1]:
import fuzzingbook_utils

In [2]:
from ConcolicFuzzer import ConcolicGrammarFuzzer, ConcolicTracer
from GrammarCoverageFuzzer import GrammarCoverageFuzzer
from Coverage import Coverage

OSError: could not find class definition

## Target Program

In [ ]:
from InformationFlow import INVENTORY_GRAMMAR, SQLException
from ConcolicFuzzer import ConcolicDB

INITIAL_DB = [
    (
        'vehicles',
        (
            {'year': int, 'kind': str, 'company': str, 'model': str},
            [
                {'year': 1997, 'kind': 'van', 'company': 'Ford', 'model': 'E350'},
                {'year': 2000, 'kind': 'car', 'company': 'Mercury', 'model': 'Cougar'},
                {'year': 1999, 'kind': 'car', 'company': 'Chevy', 'model': 'Venture'}
            ]
        )
    ),
    (
        'months',
        (
            {'month': int, 'name': str},
            [
                {'month': 1, 'name': 'jan'},
                {'month': 2, 'name': 'feb'},
                {'month': 3, 'name': 'mar'},
                {'month': 4, 'name': 'apr'},
                {'month': 5, 'name': 'may'},
                {'month': 6, 'name': 'jun'},
                {'month': 7, 'name': 'jul'},
                {'month': 8, 'name': 'aug'},
                {'month': 9, 'name': 'sep'},
                {'month': 10, 'name': 'oct'},
                {'month': 11, 'name': 'nov'},
                {'month': 12, 'name': 'dec'}
            ]
        )
    )
]

def db_select(query):
    global INITIAL_DB
    concolic_db = ConcolicDB()
    concolic_db.db = INITIAL_DB
    return concolic_db.sql(query)

## Fuzzing Configuration

In [ ]:
N_FUZZ = 100
TARGET = db_select
GRAMMAR = INVENTORY_GRAMMAR
MAX_NONTERMINALS = 5

## Fuzzer Runners

In [ ]:
def run_grammar_coverage_fuzzer(target, grammar, max_nonterminals, n_fuzz): 
    gcf = GrammarCoverageFuzzer(grammar=grammar, 
                                max_nonterminals=max_nonterminals)
    total_coverage = set()
    sql_exceptions = []
    other_exceptions = []
    
    for _ in range(n_fuzz):
        query = gcf.fuzz()
        with Coverage() as cov:
            try:
                target(query)
            except SQLException as e:
                sql_exceptions.append(e)
            except Exception as e:
                other_exceptions.append(e)
            
        total_coverage |= cov.coverage()
    
    return total_coverage, sql_exceptions, other_exceptions

In [ ]:
def run_concolic_grammar_fuzzer(target, grammar, max_nonterminals, n_fuzz):
    cgf = ConcolicGrammarFuzzer(grammar=grammar,
                                max_nonterminals=max_nonterminals)

    tokens_to_prune = ['<value>', '<table>', '<column>', 
                       '<literals>', '<exprs>', '<bexpr>']    
    cgf.prune_tokens(tokens_to_prune)
    
    total_coverage = set()
    sql_exceptions = []
    other_exceptions = []
    
    for _ in range(n_fuzz):
        query = cgf.fuzz()
        
        with ConcolicTracer() as context_manager:
            with Coverage() as cov:
                try:
                    context_manager[target](query)
                except SQLException as e:
                    sql_exceptions.append(e)
                except Exception as e:
                    other_exceptions.append(e)
                    
            cgf.update_grammar(context_manager)
            
            total_coverage |= cov.coverage()
    
    return total_coverage, sql_exceptions, other_exceptions

## Reporters

In [ ]:
from collections import defaultdict
    
def report_line_coverage(coverage):
    n_lines_covered = defaultdict(int)
    n_lines_covered['all functions'] = len(coverage)
    
    for func_name, line in coverage:
        n_lines_covered[func_name] += 1
    
    print('-' * 80)
    print(' {0: <40} {1}'.format('Function Name', 'Number of Lines Covered'))
    print('-' * 80)
    
    sorted_by_n_lines = sorted(n_lines_covered.items(), 
                               key=lambda x: x[1],
                               reverse=True)
    
    for func_name, n_lines in sorted_by_n_lines:
        print(' {0: <40} {1}'.format(func_name, n_lines))
        
    print('-' * 80 + '\n')

In [ ]:
def report_exceptions(sql_exceptions, other_exceptions):
    print('-' * 80)
    print(' {0: <40} {1}'.format('Exception Type', 'Number of Occurrences'))
    print('-' * 80)
    print(' {0: <40} {1}'.format('SQLException', len(sql_exceptions)))
    print(' {0: <40} {1}'.format('Other', len(other_exceptions)))    
    print('-' * 80 + '\n')

## Evaluation

In [ ]:
coverage, sql_exceptions, other_exceptions = run_grammar_coverage_fuzzer(
    target=TARGET, 
    grammar=GRAMMAR, 
    max_nonterminals=MAX_NONTERMINALS, 
    n_fuzz=10
)

report_line_coverage(coverage)
report_exceptions(sql_exceptions, other_exceptions)

In [ ]:
coverage, sql_exceptions, other_exceptions = run_concolic_grammar_fuzzer(
    target=TARGET, 
    grammar=GRAMMAR, 
    max_nonterminals=MAX_NONTERMINALS, 
    n_fuzz=10
)

report_line_coverage(coverage)